In [121]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split

from keras import Model , Sequential
from keras.layers import Dense

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go


In [122]:
def readData():
    #Hack to get path of Notebook (Data file is in same directory structure)
    
    if '_dh' in globals():
        current_folder = globals()['_dh'][0]
    else:
        script_path=__file__
        current_folder=dirname(script_path)

    # Calculating path to the test data
    data_location = os.path.join(current_folder,'..' ,'data' , 'data_gen.xlsx')
    data_location = os.path.normpath(data_location)

    wb = load_workbook(data_location, data_only=True)
    ws=wb['data']

    data={'x':[], 'y':[]}
    cur_row=2

    while ws['A'+str(cur_row)].value != None:
        data['x'].append(ws['A'+str(cur_row)].value)
        data['y'].append(ws['B'+str(cur_row)].value)
        cur_row += 1
    df = pd.DataFrame(data)
    train, test =  train_test_split(df, test_size=0.2)
    return train, test

In [123]:
def plotData(train, test, predictions):
    init_notebook_mode(connected=True)
    series1 = go.Scatter(
    x = train['x'],
    y = train['y'],
    name = 'Train',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgb(0,0,152)'
        )
    )

    series2 = go.Scatter(
        x = test['x'],
        y = test['y'],
        name = 'Test',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(0,152,0)'
        )
    )

    series3 = go.Scatter(
        x = test['x'],
        y = predictions.flatten(),
        name = 'Predictions',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(256,256,0)'
        )
    )
    
    data = [series1,  series3]
    
    layout = dict(title = 'Training , Test and predictions',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )
    
    fig = dict(data=data, layout=layout)
    
    iplot(fig)

In [124]:
def defineModel():
    model = Sequential()
    model.add(Dense(15, input_dim=1, activation='sigmoid'))
    model.add(Dense(350, activation='sigmoid'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [125]:
train, test=readData()

train_x = np.array(train['x'], dtype=np.float32)
train_y = np.array(train['y'], dtype=np.float32)

test_x = np.array(test['x'], dtype=np.float32)
test_y = np.array(test['y'], dtype=np.float32)

In [126]:
model = defineModel()

In [127]:
model.fit(train_x,train_y, epochs=200, verbose=2, batch_size=32,validation_data=(test_x, test_y))

Train on 8000 samples, validate on 2001 samples
Epoch 1/200
 - 2s - loss: 6682.4179 - val_loss: 3936.3626
Epoch 2/200
 - 1s - loss: 2911.3293 - val_loss: 1832.6838
Epoch 3/200
 - 1s - loss: 1151.2617 - val_loss: 678.4705
Epoch 4/200
 - 1s - loss: 509.2665 - val_loss: 356.8891
Epoch 5/200
 - 1s - loss: 284.5357 - val_loss: 223.9710
Epoch 6/200
 - 1s - loss: 199.4679 - val_loss: 134.1948
Epoch 7/200
 - 1s - loss: 140.6345 - val_loss: 264.6509
Epoch 8/200
 - 1s - loss: 108.3865 - val_loss: 63.3724
Epoch 9/200
 - 1s - loss: 72.1612 - val_loss: 48.1744
Epoch 10/200
 - 1s - loss: 65.9378 - val_loss: 43.0099
Epoch 11/200
 - 1s - loss: 44.6043 - val_loss: 29.8148
Epoch 12/200
 - 1s - loss: 41.4019 - val_loss: 39.6050
Epoch 13/200
 - 1s - loss: 35.3385 - val_loss: 51.5023
Epoch 14/200
 - 1s - loss: 32.0490 - val_loss: 59.7122
Epoch 15/200
 - 1s - loss: 29.3878 - val_loss: 15.5458
Epoch 16/200
 - 1s - loss: 37.6685 - val_loss: 33.3031
Epoch 17/200
 - 1s - loss: 31.9706 - val_loss: 12.2005
Epoch 

Epoch 152/200
 - 1s - loss: 6.4090 - val_loss: 0.8304
Epoch 153/200
 - 1s - loss: 8.4485 - val_loss: 64.3388
Epoch 154/200
 - 1s - loss: 7.0852 - val_loss: 1.2051
Epoch 155/200
 - 1s - loss: 2.7984 - val_loss: 6.2517
Epoch 156/200
 - 1s - loss: 14.8894 - val_loss: 2.1620
Epoch 157/200
 - 1s - loss: 5.3815 - val_loss: 1.0832
Epoch 158/200
 - 1s - loss: 4.0921 - val_loss: 4.6161
Epoch 159/200
 - 1s - loss: 3.9778 - val_loss: 1.0503
Epoch 160/200
 - 1s - loss: 2.8243 - val_loss: 16.3456
Epoch 161/200
 - 1s - loss: 2.6836 - val_loss: 0.7585
Epoch 162/200
 - 1s - loss: 11.8580 - val_loss: 1.2089
Epoch 163/200
 - 1s - loss: 1.6843 - val_loss: 0.7973
Epoch 164/200
 - 1s - loss: 10.5212 - val_loss: 6.5132
Epoch 165/200
 - 1s - loss: 2.5446 - val_loss: 4.1674
Epoch 166/200
 - 1s - loss: 6.9567 - val_loss: 15.6896
Epoch 167/200
 - 1s - loss: 6.7134 - val_loss: 4.8518
Epoch 168/200
 - 1s - loss: 3.0788 - val_loss: 4.0322
Epoch 169/200
 - 1s - loss: 2.6330 - val_loss: 0.9144
Epoch 170/200
 - 1s - 

In [128]:
predictions = model.predict(test_x)

In [129]:
plotData(train, test, predictions)